<a href="https://colab.research.google.com/github/JohnDiognes/analise-perfil-eleitor-vila-velha/blob/main/notebooks/01_extracao_e_limpeza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação e Importação de Bibliotecas

In [4]:
!pip install pandas numpy matplotlib seaborn requests zipfile36 py7zr

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import zipfile
import io
import os
import py7zr

pd.set_option("display.max_columns", None)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 16.7 MB/s eta 0:00:00


Download Automático dos Dados do TSE

In [5]:
import os
import requests
import zipfile
import io
import pandas as pd

# Criar pasta de dados se não existir
os.makedirs("dados", exist_ok=True)

# URL correta para o ZIP do eleitorado 2024 no Portal de Dados Abertos do TSE
url = "https://dadosabertos.tse.jus.br/dataset/eleitorado-2024/resource/e35b6053-1dea-4b9f-a4a8-12ae833bf95c/download/perfil_eleitorado_2024.zip"

print("🔽 Baixando ZIP do perfil do eleitorado 2024...")
response = requests.get(url)
if response.status_code != 200:
    print("❌ Erro no download:", response.status_code)
else:
    z = zipfile.ZipFile(io.BytesIO(response.content))
    z.extractall("dados")
    print("✔️ Arquivos extraídos na pasta dados/")

# Listar arquivos extraídos para ver o que veio
print("Arquivos extraídos:")
print(os.listdir("dados"))

# Carregar todos os CSVs extraídos (ou pelo menos os que tiverem ES)
for nome in os.listdir("dados"):
    if nome.lower().endswith(".csv"):
        caminho = os.path.join("dados", nome)
        print("Lendo:", nome)
        df = pd.read_csv(caminho, sep=";", encoding="latin1", dtype=str)
        print("Shape:", df.shape)
        print(df.head())

        # Verificar se há dados do Espírito Santo
        if "SIGLA_UF" in df.columns:
            df_es = df[df["SIGLA_UF"] == "ES"]
            print("Registros ES:", df_es.shape)



🔽 Baixando ZIP do perfil do eleitorado 2024...
✔️ Arquivos extraídos na pasta dados/
Arquivos extraídos:
['perfil_eleitor_secao_2024_ES.csv', 'leiame.pdf']
Lendo: perfil_eleitor_secao_2024_ES.csv
Shape: (1573969, 31)
   DT_GERACAO HH_GERACAO ANO_ELEICAO SG_UF CD_MUNICIPIO      NM_MUNICIPIO  \
0  13/09/2024   14:34:53        2024    ES        57053           VITÓRIA   
1  13/09/2024   14:34:53        2024    ES        56294          COLATINA   
2  13/09/2024   14:34:53        2024    ES        56774      NOVA VENÉCIA   
3  13/09/2024   14:34:53        2024    ES        57916  LARANJA DA TERRA   
4  13/09/2024   14:34:53        2024    ES        56111           ARACRUZ   

  NR_ZONA NR_SECAO NR_LOCAL_VOTACAO                      NM_LOCAL_VOTACAO  \
0       1      402             1805   EEEFM MAJOR ALFREDO PEDRO RABAYOLLI   
1       6      271             1449         EMEF PROF. JOÃO ELIAS PANCOTO   
2      30      284             1430  CMEI DOM BOSCO - LUZIA ALVES DE LIMA   
3       7   

Enviar arquivos do seu computador para o Colab

In [6]:
from google.colab import files
uploaded = files.upload()


Saving BairroXRegiao.csv to BairroXRegiao.csv
Saving Local_votacao_Vila_Velha.csv to Local_votacao_Vila_Velha.csv


Mover os arquivos para a pasta Dados

In [7]:
import shutil

# mover arquivos enviados para a pasta dados/
shutil.move("Local_votacao_Vila_Velha.csv", "dados/Local_votacao_Vila_Velha.csv")
shutil.move("BairroXRegiao.csv", "dados/BairroXRegiao.csv")


'dados/BairroXRegiao.csv'

Verificar pasta dados

In [8]:
import os

os.listdir("dados")


['Local_votacao_Vila_Velha.csv',
 'BairroXRegiao.csv',
 'perfil_eleitor_secao_2024_ES.csv',
 'leiame.pdf']

Carregar o arquivo no DataFrame

In [10]:
##import pandas as pd

##caminho = "dados/perfil_eleitor_secao_2024_ES.csv"

##df_es = pd.read_csv(caminho, sep=";", encoding="latin1", dtype=str)

##df_es.head()

import pandas as pd

# Definir o caminho da pasta onde os arquivos estão
pasta = "dados/"

# Carregar cada arquivo para um dataframe
df_local_votacao = pd.read_csv(pasta + "Local_votacao_Vila_Velha.csv", sep=",", encoding="latin1")
df_bairro_regiao = pd.read_csv(pasta + "BairroXRegiao.csv", sep=",", encoding="latin1")
df_perfil_secao = pd.read_csv(pasta + "perfil_eleitor_secao_2024_ES.csv", sep=";", encoding="latin1", low_memory=False)

# Verificar os dataframes
df_local_votacao.head(), df_bairro_regiao.head(), df_perfil_secao.head()


(   num_zon nom_localidade  num_local            nom_local  \
 0       32     VILA VELHA       1082  UMEF ANA BERNARDES    
 1       32     VILA VELHA       1082  UMEF ANA BERNARDES    
 2       32     VILA VELHA       1082  UMEF ANA BERNARDES    
 3       32     VILA VELHA       1082  UMEF ANA BERNARDES    
 4       32     VILA VELHA       1082  UMEF ANA BERNARDES    
 
                des_endereco   num_cep nom_bairro  num_secao  qtd_aptos  
 0  R. SENADOR MONJARDIM, 01  29114710    ARGOLAS         49        274  
 1  R. SENADOR MONJARDIM, 01  29114710    ARGOLAS         50        276  
 2  R. SENADOR MONJARDIM, 01  29114710    ARGOLAS         51        276  
 3  R. SENADOR MONJARDIM, 01  29114710    ARGOLAS         52        277  
 4  R. SENADOR MONJARDIM, 01  29114710    ARGOLAS         53        276  ,
    num_regiao              nom_bairro
 0           1             Boa Vista I
 1           1            Boa Vista II
 2           1                  Centro
 3           1  Coqueiral

converter colunas para número

In [11]:
cols_numericas = [
    "QT_ELEITORES_PERFIL",
    "QT_ELEITORES_BIOMETRIA",
    "QT_ELEITORES_DEFICIENCIA",
    "QT_ELEITORES_INC_NM_SOCIAL"
]

for col in cols_numericas:
    df[col] = pd.to_numeric(df[col], errors="coerce")


Filtrar somente municipio de Vila Velha

In [12]:
df_vv = df[df["NM_MUNICIPIO"] == "VILA VELHA"]
df_vv.head()



,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,NM_LOCAL_VOTACAO,CD_GENERO,DS_GENERO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,CD_RACA_COR,DS_RACA_COR,CD_IDENTIDADE_GENERO,DS_IDENTIDADE_GENERO,CD_QUILOMBOLA,DS_QUILOMBOLA,CD_INTERPRETE_LIBRAS,DS_INTERPRETE_LIBRAS,TP_OBRIGATORIEDADE_VOTO,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
5,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,32,266,1430,UMEF GRACIANO NEVES,4,FEMININO,3,CASADO,8084,80 a 84 anos,6,ENSINO MÉDIO COMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Facultativo,1,1,0,0
11,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,32,951,2070,COLÉGIO CEIC,4,FEMININO,3,CASADO,6569,65 a 69 anos,7,SUPERIOR INCOMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Obrigatório,1,1,0,0
13,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,32,936,2070,COLÉGIO CEIC,4,FEMININO,3,CASADO,7074,70 a 74 anos,7,SUPERIOR INCOMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Facultativo,1,1,1,0
17,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,57,155,1201,UMEF JUIZ JAIRO DE MATTOS PEREIRA,4,FEMININO,1,SOLTEIRO,5559,55 a 59 anos,4,ENSINO FUNDAMENTAL COMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Obrigatório,1,1,0,0
29,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,57,48,1341,CEEFMTI PASTOR OLIVEIRA DE ARAUJO- ESCOLA VIVA,2,MASCULINO,3,CASADO,6569,65 a 69 anos,6,ENSINO MÉDIO COMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Obrigatório,1,1,0,0


Visualiza dados

In [13]:
df_vv.groupby("NM_MUNICIPIO").agg({
    "QT_ELEITORES_PERFIL": "sum",
    "QT_ELEITORES_BIOMETRIA": "sum",
    "QT_ELEITORES_DEFICIENCIA": "sum"
}).reset_index()


,NM_MUNICIPIO,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA
0,VILA VELHA,345228,317542,9073


VALIDAR SE TODOS OS BAIRROS ESTÃO NO ARQUIVO

In [14]:
import pandas as pd

# Carrega as bases
df_locais = pd.read_csv("dados/Local_votacao_Vila_Velha.csv", encoding="latin1")
df_bairros = pd.read_csv("dados/BairroXRegiao.csv", encoding="latin1")

# Normalização dos nomes
bairros_locais = df_locais['nom_bairro'].str.upper().str.strip().unique()
bairros_regiao = df_bairros['nom_bairro'].str.upper().str.strip().unique()

# Conjuntos para comparação
set_locais = set(bairros_locais)
set_regiao = set(bairros_regiao)

# Bairros faltantes (presentes nos locais, mas ausentes na tabela de regiões)
bairros_faltantes = sorted(list(set_locais - set_regiao))

print("Total de bairros nos locais de votação:", len(set_locais))
print("Total de bairros no arquivo BairroXRegiao:", len(set_regiao))
print("Quantidade de bairros faltantes:", len(bairros_faltantes))

bairros_faltantes


Total de bairros nos locais de votação: 59
Total de bairros no arquivo BairroXRegiao: 94
Quantidade de bairros faltantes: 0


[]

Integrar Região ↔ Bairro nos locais de votação

In [15]:
df_bairros["nom_bairro_up"] = df_bairros["nom_bairro"].str.upper().str.strip()
df_locais["nom_bairro_up"] = df_locais["nom_bairro"].str.upper().str.strip()

df_locais_regiao = df_locais.merge(
    df_bairros[["num_regiao", "nom_bairro_up"]],
    on="nom_bairro_up",
    how="left"
)

df_locais_regiao.head()


,num_zon,nom_localidade,num_local,nom_local,des_endereco,num_cep,nom_bairro,num_secao,qtd_aptos,nom_bairro_up,num_regiao
0,32,VILA VELHA,1082,UMEF ANA BERNARDES,"R. SENADOR MONJARDIM, 01",29114710,ARGOLAS,49,274,ARGOLAS,3
1,32,VILA VELHA,1082,UMEF ANA BERNARDES,"R. SENADOR MONJARDIM, 01",29114710,ARGOLAS,50,276,ARGOLAS,3
2,32,VILA VELHA,1082,UMEF ANA BERNARDES,"R. SENADOR MONJARDIM, 01",29114710,ARGOLAS,51,276,ARGOLAS,3
3,32,VILA VELHA,1082,UMEF ANA BERNARDES,"R. SENADOR MONJARDIM, 01",29114710,ARGOLAS,52,277,ARGOLAS,3
4,32,VILA VELHA,1082,UMEF ANA BERNARDES,"R. SENADOR MONJARDIM, 01",29114710,ARGOLAS,53,276,ARGOLAS,3


Verificar se algum registro ficou sem região

In [16]:
df_locais_regiao[df_locais_regiao["num_regiao"].isna()]


,num_zon,nom_localidade,num_local,nom_local,des_endereco,num_cep,nom_bairro,num_secao,qtd_aptos,nom_bairro_up,num_regiao


Verificar nomes de colunas

In [30]:
df_vv.columns


Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'SG_UF', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO',
       'NM_LOCAL_VOTACAO', 'CD_GENERO', 'DS_GENERO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_FAIXA_ETARIA', 'DS_FAIXA_ETARIA',
       'CD_GRAU_ESCOLARIDADE', 'DS_GRAU_ESCOLARIDADE', 'CD_RACA_COR',
       'DS_RACA_COR', 'CD_IDENTIDADE_GENERO', 'DS_IDENTIDADE_GENERO',
       'CD_QUILOMBOLA', 'DS_QUILOMBOLA', 'CD_INTERPRETE_LIBRAS',
       'DS_INTERPRETE_LIBRAS', 'TP_OBRIGATORIEDADE_VOTO',
       'QT_ELEITORES_PERFIL', 'QT_ELEITORES_BIOMETRIA',
       'QT_ELEITORES_DEFICIENCIA', 'QT_ELEITORES_INC_NM_SOCIAL', 'num_zon',
       'nom_localidade', 'num_local', 'nom_local', 'des_endereco', 'num_cep',
       'nom_bairro', 'num_secao', 'qtd_aptos', 'nom_bairro_up', 'num_regiao'],
      dtype='object')

Preparar chaves de junção

In [18]:
df_vv["NR_SECAO"] = df_vv["NR_SECAO"].astype(int)
df_vv["NR_ZONA"] = df_vv["NR_ZONA"].astype(int)

df_locais_regiao["num_secao"] = df_locais_regiao["num_secao"].astype(int)
df_locais_regiao["num_zon"] = df_locais_regiao["num_zon"].astype(int)


/tmp/ipython-input-2531607740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vv["NR_SECAO"] = df_vv["NR_SECAO"].astype(int)
/tmp/ipython-input-2531607740.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vv["NR_ZONA"] = df_vv["NR_ZONA"].astype(int)


Integrar Locais_Região ↔ Perfil_Eleitor_ES

In [28]:
df_final = df_vv.merge(
    df_locais_regiao,
    left_on=["NR_ZONA", "NR_SECAO"],
    right_on=["num_zon", "num_secao"],
    how="left"
)
df_final.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,NM_LOCAL_VOTACAO,CD_GENERO,DS_GENERO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,CD_RACA_COR,DS_RACA_COR,CD_IDENTIDADE_GENERO,DS_IDENTIDADE_GENERO,CD_QUILOMBOLA,DS_QUILOMBOLA,CD_INTERPRETE_LIBRAS,DS_INTERPRETE_LIBRAS,TP_OBRIGATORIEDADE_VOTO,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL,num_zon,nom_localidade,num_local,nom_local,des_endereco,num_cep,nom_bairro,num_secao,qtd_aptos,nom_bairro_up,num_regiao
0,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,32,266,1430,UMEF GRACIANO NEVES,4,FEMININO,3,CASADO,8084,80 a 84 anos,6,ENSINO MÉDIO COMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Facultativo,1,1,0,0,32,VILA VELHA,1430,UMEF GRACIANO NEVES,"R. COUTO AGUIRRE, 49",29115050,PAUL,266,224,PAUL,3
1,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,32,951,2070,COLÉGIO CEIC,4,FEMININO,3,CASADO,6569,65 a 69 anos,7,SUPERIOR INCOMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Obrigatório,1,1,0,0,32,VILA VELHA,2070,COLÉGIO CEIC,"RUA JAIME DUARTE NASCIMENTO, 617",29101620,ITAPOÃ,951,330,ITAPOÃ,1
2,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,32,936,2070,COLÉGIO CEIC,4,FEMININO,3,CASADO,7074,70 a 74 anos,7,SUPERIOR INCOMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Facultativo,1,1,1,0,32,VILA VELHA,2070,COLÉGIO CEIC,"RUA JAIME DUARTE NASCIMENTO, 617",29101620,ITAPOÃ,936,338,ITAPOÃ,1
3,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,57,155,1201,UMEF JUIZ JAIRO DE MATTOS PEREIRA,4,FEMININO,1,SOLTEIRO,5559,55 a 59 anos,4,ENSINO FUNDAMENTAL COMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Obrigatório,1,1,0,0,57,VILA VELHA,1201,UMEF JUIZ JAIRO DE MATTOS,"RUA PASTOR AMBROSINO BARBOSA, S/ N",29114080,SÃO TORQUATO,155,247,SÃO TORQUATO,4
4,13/09/2024,14:34:53,2024,ES,57037,VILA VELHA,57,48,1341,CEEFMTI PASTOR OLIVEIRA DE ARAUJO- ESCOLA VIVA,2,MASCULINO,3,CASADO,6569,65 a 69 anos,6,ENSINO MÉDIO COMPLETO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,-1,NÃO INFORMADO,Obrigatório,1,1,0,0,57,VILA VELHA,1341,CEEFMTI PASTOR OLIVEIRA DE,"AVENIDA OTÁVIO BORIN, S/N",29111204,COBILÂNDIA,48,335,COBILÂNDIA,4


Verificar se algum registro ficou sem Região

In [20]:
df_final[df_final["nom_bairro"].isna()]


,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,NM_LOCAL_VOTACAO,CD_GENERO,DS_GENERO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,CD_RACA_COR,DS_RACA_COR,CD_IDENTIDADE_GENERO,DS_IDENTIDADE_GENERO,CD_QUILOMBOLA,DS_QUILOMBOLA,CD_INTERPRETE_LIBRAS,DS_INTERPRETE_LIBRAS,TP_OBRIGATORIEDADE_VOTO,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL,num_zon,nom_localidade,num_local,nom_local,des_endereco,num_cep,nom_bairro,num_secao,qtd_aptos,nom_bairro_up,num_regiao


Criar a Base Final

In [37]:
# ---------------------------------------------
# 1. Filtrar apenas eleitores de Vila Velha
# ---------------------------------------------
df_vv = df_final[df_final["NM_MUNICIPIO"] == "VILA VELHA"].copy()

print("Registros filtrados:", len(df_vv))

# ---------------------------------------------
# 2. Selecionar apenas colunas úteis
# (lista sugerida — ajuste conforme o que você quiser manter)
# ---------------------------------------------
colunas_uteis = [
    "ANO_ELEICAO",
    "SG_UF",
    "NM_MUNICIPIO",
    "CD_MUNICIPIO",
    "NR_ZONA", # Corrigido de NUM_ZONA
    "NR_SECAO", # Corrigido de NUM_SECAO
    # "IDADE_DATA_ELEICAO", # Removido, não existe no DF
    "CD_GENERO",
    "DS_GENERO",
    "CD_ESTADO_CIVIL",
    "DS_ESTADO_CIVIL",
    "CD_GRAU_ESCOLARIDADE",
    "DS_GRAU_ESCOLARIDADE",
    # "CD_OCUPACAO", # Removido, não existe no DF
    # "DS_OCUPACAO", # Removido, não existe no DF
    "CD_FAIXA_ETARIA",
    "DS_FAIXA_ETARIA",
    "CD_RACA_COR",
    "DS_RACA_COR",
    "CD_IDENTIDADE_GENERO",
    "DS_IDENTIDADE_GENERO",
    "CD_QUILOMBOLA",
    "DS_QUILOMBOLA",
    "CD_INTERPRETE_LIBRAS",
    "DS_INTERPRETE_LIBRAS",
    # "NUM_BAIRRO", # Removido, não existe no DF
    "nom_bairro_up", # Corrigido de NOM_BAIRRO_UP (case)
    "num_regiao", # Corrigido de NOM_REGIAO
    "NR_LOCAL_VOTACAO", # Corrigido de NUM_LOCAL
    "NM_LOCAL_VOTACAO", # Corrigido de NOM_LOCAL
    "des_endereco" # Corrigido de DES_ENDERECO (case)
]

df_vv = df_vv[colunas_uteis].copy()

# ---------------------------------------------
# 3. Renomear colunas para nomes claros e padronizados
# ---------------------------------------------
df_vv = df_vv.rename(columns={
    "ANO_ELEICAO": "ano",
    "SG_UF": "uf",
    "NM_MUNICIPIO": "municipio",
    "CD_MUNICIPIO": "cod_municipio",
    "NR_ZONA": "zona",
    "NR_SECAO": "secao",
    # "IDADE_DATA_ELEICAO": "idade", # Removido
    "CD_GENERO": "cod_genero",
    "DS_GENERO": "genero",
    "CD_ESTADO_CIVIL": "cod_estado_civil",
    "DS_ESTADO_CIVIL": "estado_civil",
    "CD_GRAU_ESCOLARIDADE": "cod_escolaridade",
    "DS_GRAU_ESCOLARIDADE": "escolaridade",
    # "CD_OCUPACAO": "cod_ocupacao", # Removido
    # "DS_OCUPACAO": "ocupacao", # Removido
    "CD_FAIXA_ETARIA": "cod_faixa_etaria",
    "DS_FAIXA_ETARIA": "faixa_etaria",
    "CD_RACA_COR": "cod_raca",
    "DS_RACA_COR": "raca",
    "CD_IDENTIDADE_GENERO": "cod_identidade_genero",
    "DS_IDENTIDADE_GENERO": "identidade_genero",
    "CD_QUILOMBOLA": "cod_quilombola",
    "DS_QUILOMBOLA": "quilombola",
    "CD_INTERPRETE_LIBRAS": "cod_interprete de libras",
    "DS_INTERPRETE_LIBRAS": "interprete de libras",
    # "NUM_BAIRRO": "cod_bairro", # Removido
    "nom_bairro_up": "bairro",
    "num_regiao": "regiao",
    "NR_LOCAL_VOTACAO": "cod_local_votacao",
    "NM_LOCAL_VOTACAO": "local_votacao_nome",
    "des_endereco": "local_votacao_endereco"
})

# ---------------------------------------------
# 4. Salvar em CSV na pasta dados/
# ---------------------------------------------
import os
os.makedirs("dados", exist_ok=True)

caminho_saida = "dados/base_vila_velha.csv"
df_vv.to_csv(caminho_saida, index=False, encoding="latin1")

print("Base final salva com sucesso em:", caminho_saida)


Registros filtrados: 179494
Base final salva com sucesso em: dados/base_vila_velha.csv


Litar colunas do arquivo final

In [39]:
list(df_vv.columns)



['ano',
 'uf',
 'municipio',
 'cod_municipio',
 'zona',
 'secao',
 'cod_genero',
 'genero',
 'cod_estado_civil',
 'estado_civil',
 'cod_escolaridade',
 'escolaridade',
 'cod_faixa_etaria',
 'faixa_etaria',
 'cod_raca',
 'raca',
 'cod_identidade_genero',
 'identidade_genero',
 'cod_quilombola',
 'quilombola',
 'cod_interprete de libras',
 'interprete de libras',
 'bairro',
 'regiao',
 'cod_local_votacao',
 'local_votacao_nome',
 'local_votacao_endereco']

Baixar os arquivos no computador

In [40]:
from google.colab import files
import os

caminho = "dados"

for arquivo in os.listdir(caminho):
    caminho_completo = os.path.join(caminho, arquivo)
    files.download(caminho_completo)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>